In [1]:
from wallet import Wallet
from accounts import SHMAccount
from ecc import PrivateKey, S256Point
from transactions import MultiSigTransaction

### Wallets


- CorporateSuperWallet is the main wallet. I AM THINKING TO MAKE MAIN WALLET A COMPANY WALLET INSTEAD, WHERE THERE IS NO PRIVATE KEYS.
- CorporateWallet: cosigners at the top corporate level.
- StoreWallet: responsible for only address generation for payments in each store.
- DailySafeWallet: Multisignature single-herarchical wallet. Manageble (see balance and create transactions) by everybody but AssistantManagerWallet who only can authorize in emergency events.
- WeeklySafeWallet: same as DailySafeWallet with the exception that no AssistantManagerWallet is part of this wallet.
- CorporateSafeWallet: Multisignature fully-herarchical wallet. Supported on the non_hardened keys within tge year account which is a hardened derived account.
- ManagerWallet: takes part in the DailySafeWallet and the WeeklySafeWallet.
- AssistantManagerWallet: only a cosigner wallet. This is the most basic and simple wallet type.


In [2]:
class CorporateSuperWallet(Wallet):
    
    def create_store_account(self, store_code):
        """
        Returns the master private key of the store with index 'store_code'.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        return 
    
    def request_pubkeys_daily_safe(self,store_code, m=2,n=6):
        """
        Returns the message to start requesting the public keys necessary to setup a
        daily safe.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction.
        store_code: Integer. The code designated to the store. It is recomended that 
        this code matches that already stablished by the company.
        """
        #save in the database the unique_id, public keys, m, n, and the store_code.
        return 
    
    def create_daily_safe_wallet(self, public_key_list, m=2, n=6,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc)
        shm_account = SHMAccount(m,n, str(daily_safes_acc.xtended_public_key), public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_weekly_safe_wallet(self,master_pubkey, public_key_list, m=2, n=5,addr_type="p2wsh",
                                 testnet=False, segwit=True):
        """
        This is a watch-only wallet by definition. CHANGE THIS TO BE ABLE TO SIGN TX!
        Returns a Single-Herarchical multisignature master wallet based on the public keys 
        and the master_pubkey.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length-1.
        """
        path = "m/44H/0H/2H"
        weekly_safes_acc = self.get_child_from_path(path)
        shm_account = SHMAccount(m,n, weekly_safes_acc.xtended_public_key, public_key_list, _privkey=None,
                                 addr_type=addr_type,testnet=testnet, segwit=segwit)
        return shm_account
    
    def create_corporate_safe(self, index):
        return 
    
    def get_store_deposit_account(self, store_index):
        return

In [3]:
class StoreWallet(Wallet):
    
     def create_daily_safe(self, date_index, public_key_list,m=2,n=6):
        """
        returns a multisignature account with the public keys and the index specified.
        date_index: Integer. Must follow YYMMDD format.
        public_key_list: the list of public keys of the keys that can sign the transaction.
        the length of the list must be 6 according to the protocol. However, a different 
        length can be stablished.
        m: minumum signatures to validate transaction. Don't modify this if not necessary.
        n: number of signatures than can sign the transaction. Must match with 
        public_key_list's length.
        """
        return 
    

In [4]:

class ManagerWallet(Wallet):
    
    def get_daily_safe_pubkey(self):
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        return daily_safes_acc.private_key_obj.point.sec()
    
    def accept_daily_safe_wallet(self,invite):
        """
        accepts the invitation from a SuperCorporateWallet to participate in a daily-safe wallet.
        The acceptance must be through a private key that matches one of the public keys that are
        part of the SHM wallet passed in the invite.
        invite: SHMAccount object. The wallet that we are going to be part of.
        """
        path = "m/44H/0H/1H"
        daily_safes_acc = self.get_child_from_path(path)
        print(daily_safes_acc.private_key)
        shm_account = SHMAccount(invite.m,invite.n, str(invite.xtended_pubkey), invite.public_keys, 
                                 daily_safes_acc.private_key_obj,invite.addr_type,invite.testnet)
        return shm_account
        

In [5]:
words = "engine over neglect science fatigue dawn axis parent mind man escape era goose border invest slab relax bind desert hurry useless lonely frozen morning"
manager1 = ManagerWallet.recover_from_words(words, 256, "RobertPauslon",True)

words = "client sudden sunset borrow pupil rely sand girl prefer movie bachelor guilt giraffe glove much strong dizzy switch ill silent goddess crumble goat power"
manager2 = ManagerWallet.recover_from_words(words,entropy=256,passphrase="RobertPaulson",testnet=True)


public_keys = [ manager1.get_daily_safe_pubkey(), manager2.get_daily_safe_pubkey(), PrivateKey(int.from_bytes(b"Ashley Serna","little")).point.sec()]


8418572761394767459444418608530371567217744336876557229650655736270882421023962287566834689847418048023007033142570964655978481626857402470209587761075589
6018716354834777059445516500463325153454936853151215624488541646839939367247233986759821493097378851875115552924255831805062990606698886137563075847103799


In [6]:
corp_wallet=CorporateSuperWallet.generate_random(testnet=True)

Copy these words for future recovery:
b'other seek spend wise milk govern conduct crunch train wrestle twist cattle'
7349618075688827342644292161792411154388397071333136974711210419399617419860687939173705085036785308619307965881391603570352784472532296452988784284606559


In [7]:
invite = corp_wallet.create_daily_safe_wallet(public_keys,n=4,testnet=True)

tprv8fVXQGqBdhmo7Lymx7JARMiyt59rg9JuuS5uxVsgqKCpQt5wSD9G6HY8uVQs5CPNV5i6qt1vWes78DmmbrKa1gncZXpaL3m71GT3RsRcWda


In [8]:
daily_wallet_manager1 = manager1.accept_daily_safe_wallet(invite)

b'\xd3\xcdG\xa0\xa1\xbf\x86V\x879\xa2\x05Q3o\xb7(\x92Z\x19\xc8bzf\xf7\x9e\xd4\xfb\xc9\xf5\xbdP'


In [9]:
daily_wallet_manager2 = manager2.accept_daily_safe_wallet(invite)

b"\xfe\x8e\xa2X\xe1\xe0\x9b\xef\xd2s\x1f;\xd5\xf3g\x88\xfd\x97\x13C/\xe3H\xdf\xb8E\xcc[$H\x1f'"


In [15]:
deposit_m1 = daily_wallet_manager1.get_deposit_account(200820)
deposit_m1.address

'tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk'

In [16]:
deposit_m2 = daily_wallet_manager2.get_deposit_account(200820)
deposit_m2.address

'tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk'

'tb1qsklg6yhka9ea46py7rk08hnkv97685tn9gjtezv2e6tudyfc7qgsxhlypz'

'tb1qsklg6yhka9ea46py7rk08hnkv97685tn9gjtezv2e6tudyfc7qgsxhlypz'

In [18]:
prev_tx_id_list = ["d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c"]
my_tx = MultiSigTransaction.create(prev_tx_id_list, [("mo3WWB4PoSHrudEBik1nUqfn1uZEPNYEc8",777)], deposit_m1, fee=None, 
                 segwit=True)

trying P2WSH
FIRST CASE: 0
transaction IS SEGWIT
transaction passed: d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c
Address: tb1qd2q0n0xcgty90fyzp4g7h0edeg32lennsd6q98v9t48mduz675nqaquxuk
trying P2WSH
tx: 05202f76c09c782ac576e12a9756b721edcc0ce7ade978e0924b693f295717a8
version: 1
tx_ins:
d629b979a1d2b2eb797c0f37adee6920fac4a77b4232baaa975c4b499e862f1c:1
tx_outs:
777:OP_DUP OP_HASH160 52903efc1004de01883ba3687be2a8ea4f6b1b19 OP_EQUALVERIFY OP_CHECKSIG
1000000:OP_0 6a80f9bcd842c857a4820d51ebbf2dca22afe6738374029d855d4fb6f05af526
locktime: 0
fee: 4096
change 95127
total 100000, diff: 0
cmds is empty. Creating new set of commands
sign_input_multisig commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x19Q\x1e"h\x82e\xe9N\x06_Q\x82TB(\xdf\xd4\xb8\xd7\xef,\xf7B\x16Z\xfeE\x01', 0, 0, 0, b"R!\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca!\x02\x17n\x

In [19]:
my_tx_signed_by2 = MultiSigTransaction.sign_received_tx(my_tx,deposit_m2)

sign_input_multisig commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x19Q\x1e"h\x82e\xe9N\x06_Q\x82TB(\xdf\xd4\xb8\xd7\xef,\xf7B\x16Z\xfeE\x01', b'0E\x02!\x00\xec\x93\xaa5\xab\xab\x8e?\xf1\xb3c\xfa\x9b\xf2c\xf1>s\x8e\xe2\x89\xe1\xb9:]\x10\x8cgL\x8dz\xba\x02 e\xa4\x8cn\x17\xe2\xe4\xa0\x82\xaf\x80\xf3d\xc0\xda\xf5\xd8\xde/}\xe1\xb9\x1cj\x02A\xb8P\x10\r\xf2T\x01', 0, 0, b"R!\x03\xfb\xb2\x9f(\xe2:\x8d\x07\xf0X68\xfa\x89\xe8e\xba\xd0-\xd1yk\xfae\xad\x95\xa5\x17v*\xc2\xca!\x02\x17n\x0c\xfd\x84]'\xce\x83)\x1fC\x1d\xda3s\x8f\xcf \x96\xa6\xf3\xf7\xc4\xab\xde\xb3p\x08\xed\xe6}!\x03\x03\xb0\x94-\xbe\xd2\xd0Nq\xab+\xa9\xbc\x05\xb9 \xf8O\xfe+\x8b\x8c-\x8c\xfd\x80*\xda`\xf6\xe8\xd1!\x03\x87\x8c5:r\x19J\xbb3\xab\xe3\xff\x82\xcfNQ\xdb\xefy{[\xbc\xac\x16+\x8b\xec\x1c)\x98h\x11T\xae"]
segwit commands: [b'0D\x02 [\xdcZ\xa1\xeb\x9c\x01\xbb\xd8\xf5\xf0\xe7z\xb5\x8e\xdep\xbe\x9fS \xdd_\x91a\xb7X\xd3`\x87\xe6\xce\x02 {!a\xfc\x1

In [20]:
my_tx_signed_by2.serialize().hex()

'010000000001011c2f869e494b5c97aaba32427ba7c4fa2069eead370f7c79ebb2d2a179b929d60100000000ffffffff0209030000000000001976a91452903efc1004de01883ba3687be2a8ea4f6b1b1988ac97730100000000002200206a80f9bcd842c857a4820d51ebbf2dca22afe6738374029d855d4fb6f05af526040047304402205bdc5aa1eb9c01bbd8f5f0e77ab58ede70be9f5320dd5f9161b758d36087e6ce02207b2161fc19511e22688265e94e065f5182544228dfd4b8d7ef2cf742165afe4501483045022100ec93aa35abab8e3ff1b363fa9bf263f13e738ee289e1b93a5d108c674c8d7aba022065a48c6e17e2e4a082af80f364c0daf5d8de2f7de1b91c6a0241b850100df254018b522103fbb29f28e23a8d07f0583638fa89e865bad02dd1796bfa65ad95a517762ac2ca2102176e0cfd845d27ce83291f431dda33738fcf2096a6f3f7c4abdeb37008ede67d210303b0942dbed2d04e71ab2ba9bc05b920f84ffe2b8b8c2d8cfd802ada60f6e8d12103878c353a72194abb33abe3ff82cf4e51dbef797b5bbcac162b8bec1c2998681154ae00000000'